# **Resilience of Delhi Road Networks to Traffic Disruptions**

---
>### **Problem Statement**:

* Road networks are complex systems that are essential for movement of people and goods.
* They are also highly interconnected, making them vulnerable to disruptions caused by accidents, natural disasters, or infrastructure failures.
* These disruptions can have a significant impact on the economy and society, and can lead to many problems.

>### **Objective of Project**:

* The objective of this project is to examine the Resilience of road networks to traffic disruptions from a network science perspective.
* This will involve analyzing the network’s structure and investigating strategies for improving the network’s robustness.

>### **Background**:

* Network science is a field of study that examines the structure and function of complex networks, with Road networks being a prime example.
*   This knowledge allows us to assess the resilience of Road networks, referring to their ability to maintain connectivity and function at an acceptable level despite disruptions.

>### **Outcome**:

* The outcome of this project will be a better understanding of the network science factors that contribute to the resilience of road networks.
* This knowledge can be used to develop strategies for improving the resilience of existing networks, and to design new Road networks that are more resilient to disruptions.

In [ ]:
import pandas as pd
import networkx as nx
from geopy.distance import geodesic
import matplotlib.pyplot as plt

routes = pd.read_csv('./GTFS/routes.txt')
stop_times = pd.read_csv('./GTFS/stop_times.txt')
stops = pd.read_csv('./GTFS/stops.txt')
trips = pd.read_csv('./GTFS/trips.txt')

## Data Preprocessing

* Data preprocessing involves cleaning, organizing, and structuring these datasets to facilitate effective analysis.

In [ ]:
routes = routes[['agency_id','route_id', 'route_long_name']]

print("Routes Data:")
routes.head()

In [ ]:
stop_times = stop_times[['trip_id', 'stop_id', 'stop_sequence']]

print("Stop Times Data:")
stop_times.head()

In [ ]:
stops = stops[['stop_code', 'stop_id', 'stop_lat' , 'stop_lon', 'stop_name']]

print("Stops Data:")
stops.head()

In [ ]:
trips = trips[['route_id', 'service_id', 'trip_id']]

print("Trips Data:")
trips.head()

In [ ]:
# Create a dictionary of route_id to route_long_name
route_name = dict(zip(routes['route_id'], routes['route_long_name']))

# Create a dictionary of trip_id to route_id
trip_to_route = dict(zip(trips['trip_id'], trips['route_id']))

## Create Directed Graph

* We create a directed graph. In this graph the Nodes represent stops and Edges are the roads connecting these stops.
* We assign attributes like Stop ID, Stop Name, Stop Latitude, Stop Longitude, Degree to these Nodes. 
* The Edges are inserted between stops. We assign attributes like Trips, Routes, and Distance between the two Stops to these Edges.

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Insert Nodes
for index, row in stops.iterrows():
    stop_id = row['stop_id']
    G.add_node(stop_id, type='stop', stop_name=row['stop_name'], stop_lat=float(row['stop_lat']), stop_lon=float(row['stop_lon']), in_degree=0, out_degree=0, degree=0)    

# Insert Edges
prev_trip_id = None
prev_stop_id = None
prev_stop_seq = 0

for _, row in stop_times.iterrows():    
    cur_trip_id = row['trip_id']
    cur_stop_id = row['stop_id']
    cur_stop_seq = int(row['stop_sequence'])
    
    if prev_stop_id is not None:   
        if prev_trip_id == cur_trip_id:
            if (prev_stop_seq + 1) == cur_stop_seq:
                if prev_stop_id != cur_stop_id:
                    route_id = trip_to_route[cur_trip_id]
                    if G.has_edge(prev_stop_id, cur_stop_id):
                        if route_id in G[prev_stop_id][cur_stop_id]['trip_dict']:
                            G[prev_stop_id][cur_stop_id]['trip_dict'][route_id] += (cur_trip_id,)
                        else:
                            G[prev_stop_id][cur_stop_id]['trip_dict'][route_id] = (cur_trip_id,)
                    else:    
                        G.add_edge(prev_stop_id, cur_stop_id, trip_dict={route_id: (cur_trip_id,)}, distance=0)
                    G.nodes[prev_stop_id]['out_degree'] += 1
                    G.nodes[prev_stop_id]['degree'] = G.nodes[prev_stop_id]['in_degree'] + G.nodes[prev_stop_id]['out_degree']
                    G.nodes[cur_stop_id]['in_degree'] += 1
                    G.nodes[cur_stop_id]['degree'] = G.nodes[cur_stop_id]['in_degree'] + G.nodes[cur_stop_id]['out_degree']  
                    
        else:
            prev_stop_seq = 0
    else:
        prev_stop_seq = 0
     
    prev_stop_id = cur_stop_id
    prev_trip_id = cur_trip_id
    prev_stop_seq = cur_stop_seq
    
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

In [ ]:
print( len(G[488][233]['trip_dict']))
print(G[488][233]['trip_dict'][142])
print( G.nodes[488]['out_degree'])
print( G.nodes[488]['in_degree'])
print( G.nodes[488]['degree'])
print(G.nodes[488])

## Degree distribution

* Degree distribution is a statistical measure that describes how the number of connections (degrees) is distributed among nodes in a network.

* The degree of a node is the number of edges connected to it. Degree distribution describes the likelihood that a randomly chosen node in the network has a certain degree.

* Understanding degree distribution helps reveal important properties of the network, such as its **connectivity, robustness, and overall structure**.

* Networks can have different degree distributions. Two common types are Normal Distribution and Power-law Distribution

In [ ]:
# Extract degrees of all nodes
degree_sequence = [G.nodes[node]['degree'] for node in G.nodes]

# Create a scatter plot
plt.scatter(range(len(degree_sequence)), degree_sequence, color='red', alpha=0.7)
plt.title('Degree Scatter Plot', fontsize=15, fontweight='bold')
plt.xlabel('Node ID', fontsize=12, fontweight='bold')
plt.ylabel('Degree', fontsize=12, fontweight='bold')
plt.show()

In [ ]:
# Plot histogram
plt.hist(degree_sequence, bins=20, color='red')
plt.title('Degree Distribution Histogram', fontsize=15, fontweight='bold')
plt.xlabel('Degree', fontsize=12, fontweight='bold')
plt.ylabel('Number of Nodes', fontsize=12, fontweight='bold')
plt.show()

## Power Law Distribution

*   Some real-world networks exhibit a power-law degree distribution, meaning that a small number of nodes have very high degrees, while the majority have relatively low degrees.

*   The power-law distribution is given by $(p_k \sim k^{-\alpha})$ and the ${\alpha}$ is its degree exponent.

*   The power-law exponent ${\alpha}$ is a parameter that characterizes the shape of the power-law distribution. A smaller α indicates a steeper decline in the distribution.

In [ ]:
import powerlaw
import matplotlib.pyplot as plt

fit = powerlaw.Fit(degree_sequence, discrete=True)
fit.plot_ccdf(color='r', linewidth=1)

plt.title('Power Law Fit for Degree Distribution', fontsize=15, fontweight='bold')
plt.xlabel('Degree (log k)', fontsize=12, fontweight='bold')
plt.ylabel('Probability Density (log pk)', fontsize=12, fontweight='bold')
plt.show()

alpha = fit.power_law.alpha
print(f"Power-law exponent (alpha): {alpha}")

In [ ]:
# Fit the power-law distribution to the data
fit = powerlaw.Fit(degree_sequence, xmin=1)

# Plot the power-law fit
fit.plot_pdf(color='r', linewidth=1)

# Customize the plot
plt.title('Power Law Fit for Degree Distribution', fontsize=15, fontweight='bold')
plt.xlabel('Degree', fontsize=12, fontweight='bold')
plt.ylabel('Probability Density', fontsize=12, fontweight='bold')
plt.legend(['Power-law Fit', 'Degree Distribution'])
plt.show()

alpha = fit.power_law.alpha
print(f"Power-law exponent (alpha): {alpha}")

## Hubs

* Node degree centrality is a measure of the importance of a node in a network based on its degree, i.e., the number of edges connected to it.
* Hubs in a network refer to nodes that have a significantly higher node degree centrality compared to other nodes. 
* They play a crucial role in maintaining connectivity and influencing the flow of information.
* The concept of hubs can be related to the **power-law distribution** by considering nodes with high degrees.
* The centrality of a node i in a network can be represented as: $$C_i = \frac{\text{{Degree of Node }} i}{\text{{Maximum Degree in the Network}}}$$
* This provides a normalized measure of how central a node is based on its degree.

In [ ]:
# Calculate node degree centrality
node_degree_centrality = {node: data['degree'] for node, data in G.nodes(data=True)}

In [ ]:
# Get the top 50 nodes based on degree centrality
dc_top_nodes = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:50]

for node in dc_top_nodes[:5]:
    print(G.nodes[node])

# Create a subgraph
dc_subgraph = G.subgraph([node for node in dc_top_nodes if G.nodes[node]['type'] == 'stop'])
dc_pos = nx.kamada_kawai_layout(dc_subgraph)

dc_node_labels = {node: data['stop_name'] for node, data in dc_subgraph.nodes(data=True)}
dc_node_colors = 'red'

# Plot the subgraph
fig, ax = plt.subplots(figsize=(30, 30))
nx.draw(dc_subgraph, pos=dc_pos, labels=dc_node_labels, with_labels=True, node_size=800, node_color=dc_node_colors, alpha=1, font_size=20, font_color='black', font_weight='bold', edge_color='black', ax=ax)
plt.title('Hubs', fontsize=40, fontweight='bold')
plt.show()

## Betweenness Centrality

* Betweenness centrality is a measure of a node's centrality in a network based on its role in facilitating the flow of information or influence between other nodes.
* Nodes with high betweenness centrality have a significant influence on the communication or interaction between other nodes in the network.
* Betweenness centrality is calculated by determining the number of shortest paths that pass through a node, relative to the total number of shortest paths in the network.
* Nodes with high betweenness centrality often act as bridges or bottlenecks in the network, as they control the flow of information between different parts of the network.

In [ ]:
from geopy.distance import geodesic

# Function to calculate distance between two points using Haversine formula
def haversine_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geodesic(coords_1, coords_2).km

# Iterate through edges and calculate distance
for edge in G.edges(data=True):
    source_node, target_node, data = edge
    source_coords = (G.nodes[source_node]['stop_lat'], G.nodes[source_node]['stop_lon'])
    target_coords = (G.nodes[target_node]['stop_lat'], G.nodes[target_node]['stop_lon'])
    
    # Calculate distance using Haversine formula
    distance = haversine_distance(*source_coords, *target_coords)
    
    # Assign distance as edge weight
    G[source_node][target_node]['distance'] = distance

In [ ]:
# Calculate node betweenness centrality based on the 'distance' attribute
node_betweenness_centrality = nx.betweenness_centrality(G, weight='distance')

In [ ]:
# Get the top 50 nodes based on betweenness centrality
bc_top_nodes = sorted(node_betweenness_centrality, key=node_betweenness_centrality.get, reverse=True)[:50]

for node in bc_top_nodes[:5]:
    print(G.nodes[node])

# Create a subgraph
bc_subgraph = G.subgraph([node for node in bc_top_nodes if G.nodes[node]['type'] == 'stop'])
bc_pos = nx.kamada_kawai_layout(bc_subgraph)

bc_node_labels = {node: data['stop_name'] for node, data in bc_subgraph.nodes(data=True)}
bc_node_colors = 'red'

# Plot the subgraph
fig, ax = plt.subplots(figsize=(30, 30))
nx.draw(bc_subgraph, pos=bc_pos, labels=bc_node_labels, with_labels=True, node_size=800, node_color=bc_node_colors, alpha=1, font_size=20, font_color='black', font_weight='bold', edge_color='black', ax=ax)
plt.title('Betweenness Centrality', fontsize=40, fontweight='bold')
plt.show()

## Disruption Scenarios

* Simulate disruptions like road closures and measure their impact on network connectivity.
* Visualize the network before and after the disruption to provide a clear representation of the changes.

>### **Road Closure**

* Road closure, resulting from construction, maintenance, new projects and infrastructure improvements, is a prolonged disruption that can last for a few days, months, or even years.
* In order to simulate this kind of disruption we will remove some Edges that are associated with the Nodes.

In [ ]:
# Create a copy of the graph for road closure scenario
G_rc = G.copy()

# Identify the top 10 hubs
top_hubs = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:10]

# Function to remove at most 3 edges associated with the top hubs
def road_closure(graph, node):
    remove_edges = []
    no_of_edges = 1
    
    for successor in graph.successors(node):
        if no_of_edges <= 3:
            remove_edges.append((node, successor))
            no_of_edges += 1
        else:
            break
            
    for edge in remove_edges:
        src = edge[0]
        destn = edge[1]
        edge_data = graph[src][destn]
        trip_dict = edge_data.get('trip_dict', {})
        no_of_trips = 0
        
        for route_id in trip_dict:
            no_of_trips += len(trip_dict[route_id])

        graph.nodes[src]['out_degree'] -= no_of_trips
        graph.nodes[src]['degree'] -= no_of_trips
        graph.nodes[destn]['in_degree'] -= no_of_trips
        graph.nodes[destn]['degree'] -= no_of_trips

        # Remove the edge
        graph.remove_edge(*edge)

# Remove edges for the top hub nodes
for hub in top_hubs:
    road_closure(G_rc, hub)

# Print updated graph information
print("Number of nodes:", G_rc.number_of_nodes())
print("Number of edges:", G_rc.number_of_edges())

In [ ]:
# Road Closure Analysis
fig, axes = plt.subplots(2, 2, figsize=(20, 20))
road_closure_degree_sequence = [G_rc.nodes[node]['degree'] for node in G_rc.nodes]

# Plot the original degree distribution scatter plot
axes[0, 0].scatter(range(len(degree_sequence)), degree_sequence, color='red', alpha=0.7)
axes[0, 0].set_title('Scatter Plot Before Road Closure Disruption', fontsize=15, fontweight='bold')
axes[0, 0].set_xlabel('Node ID', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Degree', fontsize=12, fontweight='bold')

# Plot the disrupted degree distribution scatter plot
axes[0, 1].scatter(range(len(road_closure_degree_sequence)), road_closure_degree_sequence, color='blue', alpha=0.7)
axes[0, 1].set_title('Scatter Plot After Road Closure Disruption', fontsize=15, fontweight='bold')
axes[0, 1].set_xlabel('Node ID', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Degree', fontsize=12, fontweight='bold')

# Plot the original histogram
axes[1, 0].hist(degree_sequence, bins=20, color='red', edgecolor='black')
axes[1, 0].set_title('Degree Distribution Histogram Before Road Closure Disruption', fontsize=15, fontweight='bold')
axes[1, 0].set_xlabel('Degree', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')

# Plot the original histogram
axes[1, 1].hist(road_closure_degree_sequence, bins=20, color='blue', edgecolor='black')
axes[1, 1].set_title('Degree Distribution Histogram After Road Closure Disruption', fontsize=15, fontweight='bold')
axes[1, 1].set_xlabel('Degree', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot the original hubs graph
dc_top_nodes = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:50]
dc_subgraph = G.subgraph([node for node in dc_top_nodes if G.nodes[node]['type'] == 'stop'])
dc_pos = nx.kamada_kawai_layout(dc_subgraph)
dc_node_labels = {node: data['stop_name'] for node, data in dc_subgraph.nodes(data=True)}
dc_node_colors = 'red'
dc_subgraph = G.subgraph([node for node in dc_top_nodes if G.nodes[node]['type'] == 'stop'])
dc_pos = nx.kamada_kawai_layout(dc_subgraph)
nx.draw(dc_subgraph, pos=dc_pos, labels=dc_node_labels, with_labels=True, node_size=400, node_color=dc_node_colors, alpha=1, font_size=12, font_color='black', font_weight='bold', edge_color='black', ax=axes[0])
axes[0].set_title('Hubs Before Road Closure Disruption', fontsize=15, fontweight='bold')

# Plot the disrupted hubs graph
road_closure_node_degree_centrality = {node: data['degree'] for node, data in G_rc.nodes(data=True)}
road_closure_dc_top_nodes = sorted(road_closure_node_degree_centrality, key=road_closure_node_degree_centrality.get, reverse=True)[:50]
road_closure_dc_subgraph = G_rc.subgraph([node for node in dc_top_nodes if G_rc.nodes[node]['type'] == 'stop'])
road_closure_dc_pos = nx.kamada_kawai_layout(road_closure_dc_subgraph)
road_closure_dc_node_labels = {node: data['stop_name'] for node, data in road_closure_dc_subgraph.nodes(data=True)}
road_closure_dc_node_colors = 'blue'
nx.draw(road_closure_dc_subgraph, pos=road_closure_dc_pos, labels=road_closure_dc_node_labels, with_labels=True, node_size=400, node_color=road_closure_dc_node_colors, alpha=1, font_size=12, font_color='black', font_weight='bold', edge_color='black', ax=axes[1])
axes[1].set_title('Hubs After Road Closure Disruption', fontsize=15, fontweight='bold')

plt.tight_layout()
plt.show()

>### **Malignant Accidents**

* A Road accident may temporarily close a particular route. This results in temporary cancellation of some trips to stops along the route.
* To simulate this kind of disruption, we remove some trips associated with the stops connecting the route.

In [ ]:
# Create a copy of the graph for Malignant accident scenario
G_ma = G.copy()

# Identify the top 20 hubs
top_hubs = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:20]

# Function to remove at most 5 edges associated with the top hubs
def road_accident(graph, node):
    remove_edges = []
    no_of_edges = 1
    
    for successor in graph.successors(node):
        if no_of_edges <= 5:
            remove_edges.append((node, successor))
            no_of_edges += 1
        else:
            break
    
    for edge in remove_edges:
        src = edge[0]
        destn = edge[1]
        edge_data = graph[src][destn]
        trip_dict = edge_data.get('trip_dict', {})
        no_of_trips = 0
        no_of_routes = 0
        
        # Create a list of trips to remove
        trips_to_remove = []
        for route_id in trip_dict:
            if no_of_routes < 15:
                no_of_trips += len(trip_dict[route_id])
                trips_to_remove.append(route_id)
                no_of_routes += 1
            else:
                break
        
        # Remove keys from the dictionary
        for key in trips_to_remove:
            trip_dict.pop(key, None)

        graph.nodes[src]['out_degree'] -= no_of_trips
        graph.nodes[src]['degree'] -= no_of_trips
        graph.nodes[destn]['in_degree'] -= no_of_trips
        graph.nodes[destn]['degree'] -= no_of_trips
        graph[src][destn]['trip_dict'] = trip_dict

# Remove edges for the top hub nodes
for hub in top_hubs:
    road_accident(G_ma, hub)

# Print updated graph information
print("Number of nodes:", G_ma.number_of_nodes())
print("Number of edges:", G_ma.number_of_edges())


In [ ]:
print(G.nodes[149])
print(G_ma.nodes[149])

In [ ]:
# Road Accidents Analysis
fig, axes = plt.subplots(2, 2, figsize=(20, 20))
road_accident_degree_sequence = [G_ma.nodes[node]['degree'] for node in G_ma.nodes]

# Plot the original degree distribution scatter plot
axes[0, 0].scatter(range(len(degree_sequence)), degree_sequence, color='red', alpha=0.7)
axes[0, 0].set_title('Scatter Plot Before Road Accident Disruption', fontsize=15, fontweight='bold')
axes[0, 0].set_xlabel('Node ID', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Degree', fontsize=12, fontweight='bold')

# Plot the disrupted degree distribution scatter plot
axes[0, 1].scatter(range(len(road_accident_degree_sequence)), road_accident_degree_sequence, color='blue', alpha=0.7)
axes[0, 1].set_title('Scatter Plot After Road Accident Disruption', fontsize=15, fontweight='bold')
axes[0, 1].set_xlabel('Node ID', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Degree', fontsize=12, fontweight='bold')

# Plot the original histogram
axes[1, 0].hist(degree_sequence, bins=20, color='red', edgecolor='black')
axes[1, 0].set_title('Degree Distribution Histogram Before Road Accident Disruption', fontsize=15, fontweight='bold')
axes[1, 0].set_xlabel('Degree', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')

# Plot the disrupted histogram
axes[1, 1].hist(road_accident_degree_sequence, bins=20, color='blue', edgecolor='black')
axes[1, 1].set_title('Degree Distribution Histogram After Road Accident Disruption', fontsize=15, fontweight='bold')
axes[1, 1].set_xlabel('Degree', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot the original hubs graph
dc_top_nodes = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:50]
dc_subgraph = G.subgraph([node for node in dc_top_nodes if G.nodes[node]['type'] == 'stop'])
dc_pos = nx.kamada_kawai_layout(dc_subgraph)
dc_node_labels = {node: data['stop_name'] for node, data in dc_subgraph.nodes(data=True)}
dc_node_colors = 'red'
dc_subgraph = G.subgraph([node for node in dc_top_nodes if G.nodes[node]['type'] == 'stop'])
dc_pos = nx.kamada_kawai_layout(dc_subgraph)
nx.draw(dc_subgraph, pos=dc_pos, labels=dc_node_labels, with_labels=True, node_size=400, node_color=dc_node_colors, alpha=1, font_size=12, font_color='black', font_weight='bold', edge_color='black', ax=axes[0])
axes[0].set_title('Hubs Before Road Accident Disruption', fontsize=15, fontweight='bold')

# Plot the disrupted hubs graph
road_accident_node_degree_centrality = {node: data['degree'] for node, data in G_ma.nodes(data=True)}
road_accident_dc_top_nodes = sorted(road_accident_node_degree_centrality, key=road_accident_node_degree_centrality.get, reverse=True)[:50]

for node in road_accident_dc_top_nodes[:5]:
    print(G.nodes[node])

road_accident_dc_subgraph = G_ma.subgraph([node for node in road_accident_dc_top_nodes if G_ma.nodes[node]['type'] == 'stop'])
road_accident_dc_pos = nx.kamada_kawai_layout(road_accident_dc_subgraph)
road_accident_dc_node_labels = {node: data['stop_name'] for node, data in road_accident_dc_subgraph.nodes(data=True)}
road_accident_dc_node_colors = 'blue'
nx.draw(road_accident_dc_subgraph, pos=road_accident_dc_pos, labels=road_accident_dc_node_labels, with_labels=True, node_size=400, node_color=road_accident_dc_node_colors, alpha=1, font_size=12, font_color='black', font_weight='bold', edge_color='black', ax=axes[1])
axes[1].set_title('Hubs After Road Accident Disruption', fontsize=15, fontweight='bold')

plt.tight_layout()
plt.show()

>### **Natural Disasters**

* Natural Disaster can vary significantly in duration and impact on road networks.
* Short term disruptions like heavy rains have temporary impact like delay in trips. 
* But Long term disruptions like floods may last for a few days and may result in road closures, make some stops inaccessible and cancellation of all the trips along that route.
* To simulate this kind of disruption, we remove some Nodes and make those places inaccessible.

In [ ]:
# Create a copy of the graph for road closure scenario
G_nd = G.copy()

# Identify the top 5 hubs
top_hubs = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:5]

# Function to remove the hubs
def natural_disaster(graph, node):
    remove_edges = []
    
    for successor in graph.successors(node):
        remove_edges.append((node, successor))
            
    for edge in remove_edges:
        src = edge[0]
        destn = edge[1]
        edge_data = graph[src][destn]
        trip_dict = edge_data.get('trip_dict', {})
        no_of_trips = 0
        
        for route_id in trip_dict:
            no_of_trips += len(trip_dict[route_id])

        graph.nodes[src]['out_degree'] -= no_of_trips
        graph.nodes[src]['degree'] -= no_of_trips
        graph.nodes[destn]['in_degree'] -= no_of_trips
        graph.nodes[destn]['degree'] -= no_of_trips

        # Remove the edge
        graph.remove_edge(*edge)
    # Remove the hub
    graph.remove_node(node)

# Remove the top hub nodes
for hub in top_hubs:
    natural_disaster(G_nd, hub)

# Print updated graph information
print("Number of nodes:", G_nd.number_of_nodes())
print("Number of edges:", G_nd.number_of_edges())

In [ ]:
# Natural Disaster Analysis
fig, axes = plt.subplots(2, 2, figsize=(20, 20))
natural_disaster_degree_sequence = [G_nd.nodes[node]['degree'] for node in G_nd.nodes]

# Plot the original degree distribution scatter plot
axes[0, 0].scatter(range(len(degree_sequence)), degree_sequence, color='red', alpha=0.7)
axes[0, 0].set_title('Scatter Plot Before Natural Disaster Disruption', fontsize=15, fontweight='bold')
axes[0, 0].set_xlabel('Node ID', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Degree', fontsize=12, fontweight='bold')

# Plot the disrupted degree distribution scatter plot
axes[0, 1].scatter(range(len(natural_disaster_degree_sequence)), natural_disaster_degree_sequence, color='blue', alpha=0.7)
axes[0, 1].set_title('Scatter Plot After Natural Disaster Disruption', fontsize=15, fontweight='bold')
axes[0, 1].set_xlabel('Node ID', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Degree', fontsize=12, fontweight='bold')

# Plot the original histogram
axes[1, 0].hist(degree_sequence, bins=20, color='red', edgecolor='black')
axes[1, 0].set_title('Degree Distribution Histogram Before Natural Disaster Disruption', fontsize=15, fontweight='bold')
axes[1, 0].set_xlabel('Degree', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')

# Plot the disrupted histogram
axes[1, 1].hist(natural_disaster_degree_sequence, bins=20, color='blue', edgecolor='black')
axes[1, 1].set_title('Degree Distribution Histogram After Natural Disaster Disruption', fontsize=15, fontweight='bold')
axes[1, 1].set_xlabel('Degree', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Number of Nodes', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Plot the original hubs graph
dc_top_nodes = sorted(node_degree_centrality, key=node_degree_centrality.get, reverse=True)[:50]
dc_subgraph = G.subgraph([node for node in dc_top_nodes if G.nodes[node]['type'] == 'stop'])
dc_pos = nx.kamada_kawai_layout(dc_subgraph)
dc_node_labels = {node: data['stop_name'] for node, data in dc_subgraph.nodes(data=True)}
dc_node_colors = 'red'
nx.draw(dc_subgraph, pos=dc_pos, labels=dc_node_labels, with_labels=True, node_size=400, node_color=dc_node_colors, alpha=1, font_size=12, font_color='black', font_weight='bold', edge_color='black', ax=axes[0])
axes[0].set_title('Hubs Before Natural Disaster Disruption', fontsize=15, fontweight='bold')

# Plot the disrupted hubs graph
natural_disaster_node_degree_centrality = {node: data['degree'] for node, data in G_nd.nodes(data=True)}
natural_disaster_dc_top_nodes = sorted(natural_disaster_node_degree_centrality, key=natural_disaster_node_degree_centrality.get, reverse=True)[:50]
natural_disaster_dc_subgraph = G_nd.subgraph([node for node in dc_top_nodes if node in G_nd.nodes and G_nd.nodes[node]['type'] == 'stop'])
natural_disaster_dc_pos = nx.kamada_kawai_layout(natural_disaster_dc_subgraph)
natural_disaster_dc_node_labels = {node: data['stop_name'] for node, data in natural_disaster_dc_subgraph.nodes(data=True)}
natural_disaster_dc_node_colors = 'blue'
nx.draw(natural_disaster_dc_subgraph, pos=natural_disaster_dc_pos, labels=natural_disaster_dc_node_labels, with_labels=True, node_size=400, node_color=natural_disaster_dc_node_colors, alpha=1, font_size=12, font_color='black', font_weight='bold', edge_color='black', ax=axes[1])
axes[1].set_title('Hubs After Natural Disaster Disruption', fontsize=15, fontweight='bold')

plt.tight_layout()
plt.show()


## Robustness Metrics



In [ ]:
import networkx as nx
import numpy as np

# Function to calculate and print degree-based robustness metrics
def calculate_degree_robustness_metrics(graph, name):
    print(f"\nDegree-Based Robustness Metrics for {name}:\n")
    
    # Degree sequence of the graph
    degree_sequence = [graph.degree(node) for node in graph.nodes]
    
    # Average degree
    avg_degree = np.mean(degree_sequence)
    print(f"Average Degree: {avg_degree:.5f}")

    # Standard deviation of degree
    degree_std_dev = np.std(degree_sequence)
    print(f"Standard Deviation of Degree: {degree_std_dev:.5f}")

In [ ]:
# Calculate and print degree-based robustness metrics for the original graph (G)
calculate_degree_robustness_metrics(G, "Original Graph (G)")

# Calculate and print degree-based robustness metrics for Road closures (G_rc)
calculate_degree_robustness_metrics(G_rc, "Modified Graph after Road Closures (G_rc)")

In [ ]:
# Calculate and print degree-based robustness metrics for the original graph (G)
calculate_degree_robustness_metrics(G, "Original Graph (G)")

# Calculate and print degree-based robustness metrics for Malignant Accidents (G_ma)
calculate_degree_robustness_metrics(G_ma, "Modified Graph after Malignant Accidents (G_ma)")

In [ ]:
# Calculate and print degree-based robustness metrics for the original graph (G)
calculate_degree_robustness_metrics(G, "Original Graph (G)")

# Calculate and print degree-based robustness metrics for Natural Disasters (G_nd)
calculate_degree_robustness_metrics(G_nd, "Modified Graph after Natural Disasters (G_nd)")

## Rerouting Algorithm

In [ ]:
import matplotlib.pyplot as plt

# Function to find the shortest path between two nodes in a graph
def find_shortest_path(graph, source, target):
    try:
        shortest_path = nx.shortest_path(graph, source=source, target=target, weight='distance')
        return shortest_path
    except nx.NetworkXNoPath:
        return None

In [ ]:
# Find the top 20 nodes in terms of degree in the original graph (G)
top_nodes_original = sorted(G.nodes, key=lambda x: G.degree(x), reverse=True)[:20]

# Find the top 20 nodes in terms of degree in the disrupted graph (G_rc)
top_nodes_disrupted = sorted(G_rc.nodes, key=lambda x: G_rc.degree(x), reverse=True)[:20]

# Define positions for the nodes using Kamada-Kawai layout
pos_original = nx.kamada_kawai_layout(G)
pos_disrupted = nx.kamada_kawai_layout(G_rc)

# Find shortest path between two nodes in the original graph (G)
original_shortest_path = find_shortest_path(G, source=top_nodes_original[0], target=top_nodes_original[1])

# Find shortest path between two nodes in the disrupted graph (G_rc)
disrupted_shortest_path = find_shortest_path(G_rc, source=top_nodes_disrupted[0], target=top_nodes_disrupted[1])

# Plot the original graph with only the top 20 nodes
plt.figure(figsize=(10, 10))
nx.draw(G, pos_original, nodelist=top_nodes_original, with_labels=True, font_weight='bold', node_size=500, node_color='red', font_size=8)
nx.draw_networkx_edges(G, pos_original, edgelist=[(u, v) for u, v in original_shortest_path], edge_color='black', width=2)
plt.title('Original Graph with Shortest Path (Top 20 Nodes)', fontsize=15)
plt.show()

# Plot the disrupted graph with only the top 20 nodes
plt.figure(figsize=(10, 10))
nx.draw(G_rc, pos_disrupted, nodelist=top_nodes_disrupted, with_labels=True, font_weight='bold', node_size=500, node_color='blue', font_size=8)
nx.draw_networkx_edges(G_rc, pos_disrupted, edgelist=[(u, v) for u, v in disrupted_shortest_path], edge_color='black', width=2)
plt.title('Disrupted Graph with Shortest Path (Top 20 Nodes)', fontsize=15)
plt.show()